## 此文件复现于一位大佬，源文件见[https://github.com/ageron/handson-ml2/blob/master/extra_autodiff.ipynb]

计算函数的梯度向量

$\dfrac{\partial f}{\partial x} = \displaystyle{\lim_{\epsilon \to 0}}\dfrac{f(x+\epsilon, y) - f(x, y)}{\epsilon}$ .

In [1]:
from typing import Callable


def gradients(func: Callable, vars_list, interval=0.0001):
    partial_diff = []
    func_value = func(*vars_list)
    for i in range(len(vars_list)):
        # 在这里的[:]是为了防止改变原列表，也可以用copy
        delta_var = vars_list[:]
        # delta_var = vars_list.copy()
        delta_var[i] += interval
        delta_value = func(*delta_var)
        derivative = (delta_value - func_value) / interval
        partial_diff.append(derivative)
    return partial_diff


def df_2(func, vars_list):
    return gradients(func, vars_list)


def f(x, y):
    return x * x * y + y + 2

grad = df_2(f, [3, 4])
print(f"函数f在[3, 4]点处，各一阶偏导数为{grad}")

函数f在[3, 4]点处，各一阶偏导数为[24.000400000048216, 10.000000000047748]


在已知雅可比矩阵后，可以求Hessians矩阵

In [2]:
def df_dx(x, y):
    return gradients(f, [x, y])[0]

def df_dy(x, y):
    return gradients(f, [x, y])[1]


def hessians(vars_list):
    return [gradients(df_dx, vars_list), gradients(df_dy, vars_list)]


Hessians = hessians([3, 4])
print(f"f在[3, 4]处的Hessians矩阵是{Hessians}")

f在[3, 4]处的Hessians矩阵是[[7.999999951380232, 6.000099261882497], [6.000099261882497, -1.4210854715202004e-06]]


那么如何得到精确解呢？？
我们可以使用符号计算，通过类和对象表示函数

In [3]:
from typing import Union
print("符号表示，得到的是精确解")
# 定义常数类，有值属性
class Const(object):
    
    def __init__(self, value):
        self.value = value
    
    def evaluate(self):
        return self.value
    
    def __str__(self):
        return str(self.value)
    
# 定义变量类，有名字和值属性
class Var(object):
    
    def __init__(self, name, value=0):
        self.name = name
        self.value = value
        
    def evaluate(self):
        return self.value
    
    def __str__(self):
        return self.name
    
    
class BinaryOperator(object):
    def __init__(self, a: Union[Var, Const], b: Union[Var, Const]):
        self.a = a
        self.b = b
    

class Add(BinaryOperator):
    # 执行数值操作
    def evaluate(self):
        return self.a.evaluate() + self.b.evaluate()
    # 人脑写出计算符逻辑
    def __str__(self):
        return "{} + {}".format(self.a, self.b)
    

class Mul(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() * self.b.evaluate()
    
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)


x = Var("X")
y = Var("Y")
# f(x,y) = x²y + y + 2
f = Add(Mul(Mul(x, x), y), Add(y, Const(2)))
"""
print(f)时，
1. f作为Add的一个实例，执行它的__srt__方法,此时a=Mul(Mul(x, x), y), b=Add(y, Const(2))
2. 对a,执行Mul的__str__方法，a=Mul(x, x),b=y
3. 执行Mul(x, x)的__str__方法，此时为(x) * (x),返回上一层Mul,则Mul(Mul(x, x), y)结果为((x) * (x)) * (y)
4. 同理对于Add(y, Const(2))实例，嵌套得出最终结果
5. ((x) * (x)) * (y) + y + 2

"""
print(f)

# 赋值
x.value = 3
y.value = 4
# 同样的嵌套计算
res = f.evaluate()
# 函数在(3, 4)处的点
print(res)


符号表示，得到的是精确解
((X) * (X)) * (Y) + Y + 2
42


In [4]:
print("精确计算雅可比矩阵和黑塞矩阵")
# 接下来用符号解表示雅可比和Hessians，直接对类加属性
# 由于是类中的函数，所以第一个要传入self参数
Const.gradients = lambda self, var: Const(0) # 常数导数为0
# if self则是对传入的实例进行判断，是不是属于Var的对象
Var.gradients = lambda self, var: Const(1) if self is var else Const(0)
# + 法的导数，分别求导相加
Add.gradients = lambda self, var: Add(self.a.gradients(var), self.b.gradients(var))
# 乘法导数，前导后不导，后导前不导
Mul.gradients = lambda self, var: Add(Mul(self.a, self.b.gradients(var)), Mul(self.a.gradients(var), self.b))

x = Var(name="x", value=3.)
y = Var(name="y", value=4.)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

dfdx = f.gradients(x)  # 2xy
dfdy = f.gradients(y)  # x² + 1
print(dfdx)
print(dfdy)
# 此时dfdx是Add或其他类的对象
a, b = dfdx.evaluate(), dfdy.evaluate()
print(a, b)


# 计算Hessians矩阵
d2fdxdx = dfdx.gradients(x) # 2y
d2fdxdy = dfdx.gradients(y) # 2x
d2fdydx = dfdy.gradients(x) # 2x
d2fdydy = dfdy.gradients(y) # 0
Hessians = [[d2fdxdx.evaluate(), d2fdxdy.evaluate()],
 [d2fdydx.evaluate(), d2fdydy.evaluate()]]
print(Hessians)

# 常数导数为0
c = Const(2)
print(c.gradients(c.value))

精确计算雅可比矩阵和黑塞矩阵
((x) * (x)) * (0) + ((x) * (1) + (1) * (x)) * (y) + 0 + 0
((x) * (x)) * (1) + ((x) * (0) + (0) * (x)) * (y) + 1 + 0
24.0 10.0
[[8.0, 6.0], [6.0, 0.0]]
0


计算反向微分
基于链式法则
其中，对应的值由前向传播算出

In [8]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        pass
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
        self.gradient = 0
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        self.gradient += gradient
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

In [9]:
x = Var("x", init_value=3)
y = Var("y", init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y +2

result = f.evaluate()
f.backpropagate(1.0)

In [10]:
print(f)

((x) * (x)) * (y) + y + 2


In [11]:
print(result)

42


In [12]:
print(x.gradient)
print(y.gradient)

24.0
10.0


## 使用TensorFlow进行自动微分

In [14]:
import tensorflow as tf

In [15]:
# 定义变量
x = tf.Variable(3.)
y = tf.Variable(4.)

with tf.GradientTape() as tape:
    f = x*x*y + y + 2
# 计算雅可比矩阵
jacobians = tape.gradient(f, [x, y])
print(jacobians)

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


我们可以计算二阶导数

In [16]:
with tf.GradientTape(persistent=True) as tape:
    f = x*x*y + y + 2
    df_dx, df_dy = tape.gradient(f, [x, y])

d2f_d2x, d2f_dydx = tape.gradient(df_dx, [x, y])
d2f_dxdy, d2f_d2y = tape.gradient(df_dy, [x, y])
del tape

hessians = [[d2f_d2x, d2f_dydx], [d2f_dxdy, d2f_d2y]]
print(hessians)

[[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>], [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]]


计算张量相对于它不依赖的变量的导数时，`gradient()` 函数返回 `None`，而不是 0.0。